In [22]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV

import statsmodels.api as sm

In [23]:
train = pd.read_csv('../../cleaned_datasets/train.csv')
test = pd.read_csv('../../cleaned_datasets/test.csv')



In [24]:
X = train.drop(columns='saleprice')
y = train['saleprice']


***
## Linear Regression With Cross Val and Log of Saleprice for Training
***

In [25]:
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
high_p_vals = list(model.pvalues[model.pvalues >= .05].index)[2:]
X.drop(columns = high_p_vals, inplace = True)
X.drop(columns = 'const', inplace = True)


C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\statsmodels\base\model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)


In [27]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

In [28]:
lr = RidgeCV(alphas= np.logspace(0, 3, 100), scoring= 'r2', cv= 5)

y_train_log = y_train.map(np.log)
y_test_log = y_test.map(np.log)



print(cross_val_score(lr, X_train_sc, y_train_log, cv=10).mean()
)

lr.fit(X_train, y_train_log)

print(lr.score(X_train, y_train_log), lr.score(X_test, y_test_log))



0.8605684943501947
0.8890947741384252 0.8855464883259838


In [29]:

scaler.fit(X)

X_sc = scaler.transform(X)

lr.fit(X, y.map(np.log))

RidgeCV(alphas=array([   1.        ,    1.07226722,    1.149757  ,    1.23284674,
          1.32194115,    1.41747416,    1.51991108,    1.62975083,
          1.7475284 ,    1.87381742,    2.009233  ,    2.15443469,
          2.3101297 ,    2.47707636,    2.65608778,    2.84803587,
          3.05385551,    3.27454916,    3.51119173,    3.76493581,
          4.03701726,    4.32876128,    4.64158883,    4.97702356,
          5.33669923,    5.72236766,    6.13590727,    6.57933225,
          7.05480231,    7.56463328,    8.11130831,    8...
        200.92330026,  215.443469  ,  231.01297001,  247.7076356 ,
        265.60877829,  284.80358684,  305.38555088,  327.45491629,
        351.11917342,  376.49358068,  403.70172586,  432.87612811,
        464.15888336,  497.70235643,  533.66992312,  572.23676594,
        613.59072734,  657.93322466,  705.48023107,  756.46332755,
        811.13083079,  869.74900262,  932.60334688, 1000.        ]),
        cv=5, fit_intercept=True, gcv_mode=None, nor

In [30]:

testing = test.drop(columns = 'saleprice')

testing.drop(columns = high_p_vals, inplace = True)

display(test.head()
)

output = pd.DataFrame({'Id': test['id']})


output['SalePrice'] = lr.predict(testing)


output['SalePrice'] = output['SalePrice'].map(np.exp)

output['Id'] = [int(val) for val in output['Id']]

display(output)

pd.DataFrame.to_csv(output, '../../submissions/log_ridge.csv', index = False)

,index,id,pid,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,...,misc_feature_TenC,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_VWD,sale_type_WD
0,0.0,2658.0,902301120.0,190.0,69.000000,9142.0,6.0,8.0,1910.0,1950.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,2718.0,905108090.0,90.0,82.786765,9662.0,5.0,4.0,1977.0,1977.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2.0,2414.0,528218130.0,60.0,58.000000,17104.0,7.0,5.0,2006.0,2006.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3.0,1989.0,902207150.0,30.0,60.000000,8520.0,5.0,6.0,1923.0,2006.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4.0,625.0,535105100.0,20.0,79.027933,9500.0,6.0,5.0,1963.0,1963.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,Id,SalePrice
0,2658,106943.497375
1,2718,158699.176764
2,2414,216149.909082
3,1989,103728.488830
4,625,182406.767613
...,...,...
873,1662,192011.372165
874,1234,205734.090675
875,1373,126297.669053
876,1672,112610.503165
